# Machine Learning in Python - Project 1.1 - 英国重大交通事故预测
#### Topic and idea from: Achyut Kafle
#### Data from: UK Department for Transport

# 简介 Introduction
在英国，每年有数千人死于交通事故，同时有数十万人受到不同程度，甚至永久性的伤害。除了人员伤亡之外，道路交通事故还会造成时间、经济与心理损失，以及对非直接受害人造成的其他损失（比如说交通拥堵）。因此，了解导致交通事故的主要因素有助于政府制定政策以预防或减少事故，从而挽救与这些事故相关的损失。

# 目的 Goal
* 找到事故发生的决定性因素
* 尝试根据已知环境预测交通事故的严重性

# 数据 Data
本次我们将使用 UK Department for Transport 提供的开源数据。这个数据集包括以下数据：
* 时间和地点
* 车祸中牵连的汽车数量
* 道路与天气情况
* 死亡人数

## Set-up the project

In [ ]:
# Download data
!wget https://raw.githubusercontent.com/skyu0221/online-dropbox/master/ml/capstone1/project1/UK_RoadSafety_Accidents.csv

In [ ]:
# General 
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as stats

# Features pre-processing and principal component analysis (pca) 
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

# Train-test split
from sklearn.model_selection import train_test_split

# Classifiers 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier

# Classifiers ensembling
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from mlxtend.classifier import StackingClassifier

# Classifiers evaluation metrics
from sklearn.metrics import accuracy_score, roc_auc_score, auc, roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef

# Random resampling
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Tuning hyperparameters
from sklearn.model_selection import RandomizedSearchCV

# Other
from time import time
from scipy.stats import ttest_ind

# Ploting
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
from IPython.display import display
pd.options.display.float_format = '{:.3f}'.format

# Suppressing annoying harmless error
import warnings
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [ ]:
import sys

python_print = print

def print(*objects, sep=' ', end='\n', file=sys.stdout, color=(), fit_len=-1):
    line = "".join(map(str, objects))
    fit_len = max(fit_len - len(line), 0)
    for c in color:
        if len(objects) > 1:
            objects = (f"\033[{c}m{objects[0]}",) + objects[1:-1] + (
                f"{objects[-1]}{' ' * fit_len}\033[0m",)
        elif len(objects) == 1:
            objects = (f"\033[{c}m{objects[0]}{' ' * fit_len}\033[0m",)
    python_print(*objects, sep=sep, end=end, file=file)

## 数据预处理 Preprocessing

In [ ]:
# Import dataset
df=pd.read_csv('UK_RoadSafety_Accidents.csv')

# Print info
df.info()

### 数据清理 Data-cleaning

In [ ]:
# Find how many entry is missing in each feature
print(df.isnull().sum())

In [ ]:
# LSOA_of_Accident_Location stands for "Lower Super Ouput Area of Accident_Location (England & Wales only)", therefore not very useful
df = df.drop(['LSOA_of_Accident_Location'], axis=1)
df = df.dropna(axis=0, how='any')

### 数据检视 Data analysis

#### 数据聚类 Data aggregation
* "Accident_Severity": Fatal (1) / Serious (2) / Slight (3)

    * -> "Serious_Accident": Serious/Fatal (1) vs. Slight (0)

In [ ]:
df['Serious_Accident'] = 1
df['Serious_Accident'][df['Accident_Severity'] == 3] = 0
print('Count of outcome variable: \n', df['Serious_Accident'].value_counts())

# Count plot 
plt.figure(figsize=(9,6))
ax = sns.countplot(x=df.Serious_Accident)
for p in ax.patches:
    x = p.get_bbox().get_points()[:,0]
    y = p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100. * y / df['Serious_Accident'].value_counts().sum()), (x.mean(), y),
                ha='center', va='bottom') # set the alignment of the text
plt.show()

#### 数据观察
**Latitude & longitude**

In [ ]:
# Split samples for plotting
slight = df[df['Serious_Accident'] == 0]
serious = df[df['Serious_Accident'] == 1]

In [ ]:
plt.figure(figsize=(10,7))

# Latitude
slight.Latitude.plot.density(label="Slight accidents")
serious.Latitude.plot.density(label="Fatal/serious accidents")
plt.legend()
plt.title('Distribution plot: Latitude by the serverity of accidents')
plt.xlim((49, 59))
plt.xlabel("Latitude")
plt.show()

In [ ]:
plt.figure(figsize=(10,7))

# Longitude
slight.Longitude.plot.density(label="Slight accidents")
serious.Longitude.plot.density(label="Fatal/serious accidents")
plt.legend()
plt.title('Distribution plot: Longitude by the serverity of accidents')
plt.xlim((-6, 2))
plt.xlabel("Longitude")
plt.show()

**Number of vehicles involved**

In [ ]:
# Count and descriptive statistics
print("Statistics about number of vehicles involved in each accident", color=[93])
print(df['Number_of_Vehicles'].describe())
print()
print("Number of accidents for different number of vehicles", color=[92])
count = df.groupby('Number_of_Vehicles')['Accident_Index'].count().to_frame()
count["Percentage"] = count / len(df) * 100
print(count)

In [ ]:
# To simplify the task, ignore accidents with more than 4 cars
df = df.drop(df[df.Number_of_Vehicles > 4].index)

In [ ]:
plt.figure(figsize=(10,7))
ax = sns.countplot(x=df.Number_of_Vehicles, hue=df.Serious_Accident)
for p in ax.patches:
    x = p.get_bbox().get_points()[:,0]
    y = p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100. * y / df['Serious_Accident'].value_counts().sum()), (x.mean(), y), 
                ha='center', va='bottom') # set the alignment of the text
plt.xlabel("Number of vehicle involved")
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

**Number of casualties**

In [ ]:
# Count and descriptive statistics
print("Statistics about number of casualties involved in each accident", color=[93])
print(df['Number_of_Casualties'].describe())
print()
print("Number of accidents for different number of casualties", color=[92])
count = df.groupby('Number_of_Casualties')['Accident_Index'].count().to_frame()
count["Percentage"] = count / len(df) * 100
print(count)

In [ ]:
# To simplify the task, ignore accidents with more than 4 casualties
df = df.drop(df[df.Number_of_Casualties > 4].index)

In [ ]:
plt.figure(figsize=(10,7))
ax = sns.countplot(x=df.Number_of_Casualties, hue=df.Serious_Accident)
for p in ax.patches:
    x = p.get_bbox().get_points()[:,0]
    y = p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100. * y / df['Serious_Accident'].value_counts().sum()), (x.mean(), y), 
            ha='center', va='bottom') # set the alignment of the text
plt.xlabel("Number of casualties")
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

**<center>========================= Categorical variables =========================</center>**

In [ ]:
# Chi-squared test of independence
# https://en.wikipedia.org/wiki/Chi-squared_test
def chi_sq_test(col1, col2):
    count_table = pd.crosstab(df[col1], df[col2])
    print(count_table)
    print(stats.chisquare(count_table, axis=None))

**Day of week**

In [ ]:
# Day of week
mapper = {1: "Sunday", 2: "Monday", 3: "Tuesday", 4: "Wednesday", 5: "Thursday", 6: "Friday", 7: "Saturday"}
df['Day_of_Week'] = [mapper[i] for i in df['Day_of_Week']]

In [ ]:
plt.figure(figsize=(12,6))
# Count plot  
sns.countplot(x=df['Day_of_Week'], hue="Serious_Accident", data=df, order=[mapper[i] for i in mapper])
plt.xlabel('Day of week')
plt.title('Count plot: Day of week and severity of accidents')
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

# Table of counts and chi-squared test of independence
# p-value less than 0.05, reject null
print('Chi-squared test of independence between days of week and severity of accidents', color=[91])
print()
chi_sq_test('Serious_Accident','Day_of_Week')

**Month of year**

In [ ]:
# Split date into month
df['month'] = pd.DatetimeIndex(df['Date']).month

# Month of year
mapper = {1: "JAN", 2: "FEB", 3: "MAR", 4: "APR", 5: "MAY", 6: "JUN",
          7: "JUL", 8: "AUG", 9: "SEP", 10: "OCT", 11: "NOV", 12: "DEC"}
df['month'] = [mapper[i] for i in df['month']]

In [ ]:
plt.figure(figsize=(12,6))
# Count plot  
sns.countplot(x=df['month'], hue="Serious_Accident", data=df, order=[mapper[i] for i in mapper])
plt.xlabel('Month of year')
plt.title('Count plot: Month of year and severity of accidents')
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

# Table of counts and chi-squared test of independence
print('Chi-squared test of independence between months of year and severity of accidents', color=[91])
print()
chi_sq_test('Serious_Accident','month')

**Day of month**

In [ ]:
# Split day from date
df['day'] = pd.DatetimeIndex(df['Date']).day

# Day of month
df['day'] = [f"Day {i}" for i in df['day']]

In [ ]:
plt.figure(figsize=(12,6))
# Count plot  
sns.countplot(x=df['day'], hue="Serious_Accident", data=df, order=[f"Day {i + 1}" for i in range(31)])
plt.xlabel('Day of month')
plt.xticks(rotation='45')
plt.title('Count plot: Day of month and severity of accidents')
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

# Table of counts and chi-squared test of independence
print('Chi-squared test of independence between days of month and severity of accidents', color=[91])
print()
chi_sq_test('Serious_Accident','day')

**Hour of day**

In [ ]:
# Hour of day
df['hour'] = pd.DatetimeIndex(df['Time']).hour

df['hour'] = [str(i) for i in df['hour']]

In [ ]:
plt.figure(figsize=(12,6))
# Count plot
sns.countplot(x=df['hour'], hue="Serious_Accident", data=df, order=[str(i) for i in range(24)])
plt.xlabel('Hour of day')
plt.title('Count plot: Hour of day and severity of accidents')
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

# Table of counts and chi-squared test of independence
print('Chi-squared test of independence between hours of day and severity of accidents', color=[91])
print()
chi_sq_test('Serious_Accident','hour')

**First road class**

In [ ]:
# Road class 
mapper = {1: "Class A", 2: "Class A", 3: "Class A", 4: "Class B", 5: "Class C"}
df['1st_Road_Class'] = [mapper[i] if i in mapper else "Unclassified" for i in df['1st_Road_Class']]

In [ ]:
plt.figure(figsize=(12,6))
# Count plot
sns.countplot(x=df['1st_Road_Class'], hue="Serious_Accident", data=df, order=["Class A", "Class B", "Class C", "Unclassified"])
plt.xlabel('Road class')
plt.title('Count plot: Road class and severity of accidents')
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

# Table of counts and chi-squared test of independence
print('Chi-squared test of independence between first road classes and severity of accidents', color=[91])
print()
chi_sq_test('Serious_Accident','1st_Road_Class')

**Road type**

In [ ]:
# Drop missing coded as -1
df = df.drop(df[df.Road_Type == -1].index)

# Road Type
mapper = {3: "Dual Carriage Way", 6: "Single Carriage Way"}
df['Road_Type'] = [mapper[i] if i in mapper else 'Road Type Other' for i in df['Road_Type']]

In [ ]:
plt.figure(figsize=(12,6))
# Count plot
sns.countplot(x=df['Road_Type'], hue="Serious_Accident", data=df)
plt.xlabel('Road Type')
plt.title('Count plot: Road type and severity of accidents')
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

# Table of counts and chi-squared test of independence
print('Chi-squared test of independence between road types and severity of accidents', color=[91])
print()
chi_sq_test('Serious_Accident','Road_Type')

**Speed limit**

In [ ]:
# Speed limit
df['Speed_limit'] = [f"Limit {int(i)}" for i in df['Speed_limit']]

In [ ]:
plt.figure(figsize=(12,6))
# Count plot
sns.countplot(x=df['Speed_limit'], hue="Serious_Accident", data=df)
plt.xlabel('Speed limit')
plt.title('Count plot: Speed limit and severity of accidents')
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

# Table of counts and chi-squared test of independence
print('Chi-squared test of independence between road types and severity of accidents', color=[91])
print()
chi_sq_test('Serious_Accident','Speed_limit')

**Road junction**

In [ ]:
# Road junction
mapper = {0: "Within 20m", 1: "Roundabout", 2: "Roundabout", 3: "T section", 6: "Crossroads"}
df['Junction_Detail'] = [mapper[i] if i in mapper else 'Other_Junction' for i in df['Junction_Detail']]

In [ ]:
plt.figure(figsize=(12,6))
# Count plot
sns.countplot(x=df['Junction_Detail'], hue="Serious_Accident", data=df)
plt.xlabel('Road junction')
plt.title('Count plot: Road junctions and severity of accidents')
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

# Table of counts and chi-squared test of independence
print('Chi-squared test of independence between road junctions and severity of accidents', color=[91])
print()
chi_sq_test('Serious_Accident','Junction_Detail')

**Pedestrian crossing - physical facilities**

In [ ]:
# Pedestrian_Crossing-Physical_Facilities
mapper = {0: "No Facility", 5: "Traffic Signal"}
df['PedCross_PhysFacs'] = [mapper[i] if i in mapper else 'Others' for i in df['Pedestrian_Crossing-Physical_Facilities']]

In [ ]:
plt.figure(figsize=(12,6))
# Count plot
sns.countplot(x=df['PedCross_PhysFacs'], hue="Serious_Accident", data=df)
plt.xlabel('Pedestrian crossing physical facilities')
plt.title('Count plot: Pedestrian crossing physical facilities and severity of accidents')
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

# Table of counts and chi-squared test of independence
print('Chi-squared test of independence between pedestrian physical crossing facilities and severity of accidents', color=[91])
print()
chi_sq_test('Serious_Accident','PedCross_PhysFacs')

**Light conditions**

In [ ]:
# Light conditions
mapper = {1: "Daylight", 4: "Dark - Light Lit"}
df['Light_Conditions'] = [mapper[i] if i in mapper else 'Other_LightConditions' for i in df['Light_Conditions']]

In [ ]:
plt.figure(figsize=(12,6))
# Count plot
sns.countplot(x=df['Light_Conditions'], hue="Serious_Accident", data=df)
plt.xlabel('Light conditions')
plt.title('Count plot: Light conditions and severity of accidents')
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

# Table of counts and chi-squared test of independence
print('Chi-squared test of independence between road light conditions and severity of accidents', color=[91])
print()
chi_sq_test('Serious_Accident','Light_Conditions')

**Road surface conditions**

In [ ]:
# Drop missing data coded as -1
df = df.drop(df[df.Road_Surface_Conditions == -1].index)

# Road conditions
mapper = {1: "Dry", 2: "Wet", 5: "Wet"}
df['Road_Surface_Conditions']=[mapper[i] if i in mapper else 'Other' for i in df['Road_Surface_Conditions']]

In [ ]:
plt.figure(figsize=(12,6))
# Count plot
sns.countplot(x=df['Road_Surface_Conditions'], hue="Serious_Accident", data=df)
plt.xlabel('Road surface conditions')
plt.title('Count plot: Light conditions and severity of accidents')
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

# Table of counts and chi-squared test of independence
print('Chi-squared test of independence between road surface conditions and severity of accidents', color=[91])
print()
chi_sq_test('Serious_Accident','Road_Surface_Conditions')

**Urban rural classification of accident location**

In [ ]:
# Urban
df["Urban"] = ["Urban" if i == 1 else 'Rural' for i in df['Urban_or_Rural_Area']]

In [ ]:
plt.figure(figsize=(12,6))
# Count plot
sns.countplot(x=df['Urban'], hue="Serious_Accident", data=df)
plt.xlabel('Urban rural classification')
plt.title('Count plot: Rural/Urban and severity of accidents')
plt.legend(labels=['Slight accidents', 'Serious/Fatal accidents'])
plt.show()

# Table of counts and chi-squared test of independence
print('Chi-squared test of independence between rural/urban and severity of accidents', color=[91])
print()
chi_sq_test('Serious_Accident','Urban')

## Feature engineering

In [ ]:
# Convert categirical variables into present/absent or dummy variables
df = pd.get_dummies(df, columns=['Day_of_Week', 'month', 'day', 'hour', '1st_Road_Class', 'Road_Type', 
                                 'Speed_limit','Junction_Detail', 'PedCross_PhysFacs', 'Light_Conditions',
                                 'Road_Surface_Conditions', 'Urban'])

In [ ]:
# Keep relevant df features only
df = df.drop(['Accident_Index','Location_Easting_OSGR', 'Location_Northing_OSGR', 'Police_Force', 'Accident_Severity',
              'Date','Time','Local_Authority_(District)','Local_Authority_(Highway)','1st_Road_Number',
              'Junction_Control', '2nd_Road_Class','2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
              'Pedestrian_Crossing-Physical_Facilities','Special_Conditions_at_Site','Carriageway_Hazards',
              'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident','Weather_Conditions'], axis=1)

### Heatmap of correlation matrix

In [ ]:
# Correlation heatmap
corrmat = df.corr()
plt.subplots(figsize=(20,20))
mask = np.zeros_like(corrmat, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corrmat, mask=mask, vmax=0.9, cmap="YlGnBu", square=True, cbar_kws={"shrink": .5})
plt.show()

### Seperate features and labels

In [ ]:
# Seperate label (y) and features (x)
y = df.loc[:,'Serious_Accident']
x = df.loc[:, df.columns != 'Serious_Accident']
print('Shape of y: \n', y.shape[0])
print('Shape of x: \n', x.shape)

## Feature 筛选

### Principal component analysis (PCA)
Principal components analysis，(PCA）是一种统计分析、简化数据集的方法。它利用正交变换来对一系列可能相关的变量的观测值进行线性变换，从而投影为一系列线性不相关变量的值，这些不相关变量称为主成分（Principal Components）。具体地，主成分可以看做一个线性方程，其包含一系列线性系数来指示投影方向。PCA对原始数据的正则化或预处理敏感（相对缩放）。

**基本思想**：
+ 将坐标轴中心移到数据的中心，然后旋转坐标轴，使得数据在C1轴上的方差最大，即全部n个数据个体在该方向上的投影最为分散。意味着更多的信息被保留下来。C1成为第一主成分。
+ C2第二主成分：找一个C2，使得C2与C1的协方差（相关系数）为0，以免与C1信息重叠，并且使数据在该方向的方差尽量最大。
+ 以此类推，找到第三主成分，第四主成分。。。。第p个主成分。p个随机变量可以有p个主成分

In [ ]:
# Standardizing the features
x_stand = StandardScaler().fit_transform(x)

# PCA to keep 95% of variance
pca = PCA(0.95)

# Components required to keep 95% of variance
x_pca = pca.fit_transform(x_stand)

# Cumulative sum of explained variance by the components
var_cumsum = pca.explained_variance_ratio_.cumsum()
print("Cumulative Sum of variance when increase number of components", color=[91])
print(var_cumsum)
print()
print(f'Need to keep {len(var_cumsum)} components to explain 95% of the variance.', color=[43])

In [ ]:
plt.figure(figsize=(6,6))
pd.DataFrame(list(pca.explained_variance_ratio_)).plot()
plt.xlabel('Number of components')
plt.ylabel('Variance explained')
plt.show()

**Notes:**

* 根据 PCA 的结果，我们要保留 86 个 feature 才能 retain 95% variance。也就是说我们只能减少 20 个 feature（本来总共有 106 个 feature）。由于 86 个 feature 还是很多，所以我们用 K Best 来进一步缩小 feature 数量。
* 我们会稍后用 K Best 的结果与用全部 feature 的结果进行比较

### K best features

In [ ]:
# Select k best features
k = 15
x_kbest = x[x.columns[SelectKBest(f_classif, k=k).fit(x, y).get_support()]]
print("K best features:", color=[93])
print("\n".join(list(x_kbest.columns)))
print('Shape: ', x_kbest.shape, color=[91])

In [ ]:
# Heatmap with k best features
df_kbest = pd.concat([y, x_kbest], axis=1)
corrmat = df_kbest.corr()
plt.subplots(figsize=(15,12))
mask = np.zeros_like(corrmat, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corrmat, annot=True, mask=mask, vmax=0.9, cmap="YlGnBu", square=True, cbar_kws={"shrink": .5})
plt.show()

In [ ]:
# Final samples
print('Number of samples: ', df.shape[0], color=[43])
print('Proportion of positive outcomes: ', round(df['Serious_Accident'].mean(), 2), color=[43])

**Note**:

* 由于 positive outcome 只有 17%，所以我们的 sample 是 imbalance 的。我们需要将 sample 调整成 50/50。

## Split train and test samples

In [ ]:
# Split samples into train (80%) and test (20%) samples
X_train, X_test, y_train, y_test = train_test_split(x_kbest, y, test_size=0.2, random_state=321)
print('Train samples: ', X_train.shape)
print('Test samples: ', X_test.shape)

### Balance the sample

In [ ]:
# Random under sampling of majority class
rus = RandomUnderSampler(random_state=321)

X_train_rus, y_train_rus = rus.fit_sample(X_train, y_train)
idx = rus.sample_indices_
print('Random under sampling', Counter(y_train_rus))

In [ ]:
# Define a function to fit, predict and then print outputs
def fit_eval_clf(model):
    start = time()
    #Fit the model in train samples
    prediction = model.fit(X_train_rus, y_train_rus).predict(X_test)
    
    # Cross-validated accuracy score
    accu_train = cross_val_score(model, X_train_rus, y_train_rus, cv=10, scoring='accuracy')
    print(f"Cross-validated accuracy score on train samples: {accu_train.mean():.3} (+/-{accu_train.std() * 2:.3})")
    
    # Cros-validated area under ROC curve 
    au_roc_train = cross_val_score(model, X_train_rus, y_train_rus, cv=10, scoring='roc_auc')
    print (f"Cross-validated area under ROC curve on train samples: {au_roc_train.mean()} (+/- {au_roc_train.std() * 2})")
    print('\n')

    # Accuracy score on test samples
    print (f"Accuracy score on test samples: ", round(accuracy_score(y_test, prediction), 4))
    
    # Area under ROC on test samples
    print (f"Area under ROC curve on test samples: ", round(roc_auc_score(y_test, prediction), 4))
    print()
    print(f"Total time to run: {(time() - start) / 60:.2} minutes.")

In [ ]:
def rand_search_cv(model, params):
    start = time()
    # Tune the hyperparameters via a randomized search
    rgrid = RandomizedSearchCV(model, params, random_state=321, scoring='roc_auc')
    rgrid.fit(X_train_rus, y_train_rus)
    print(f"Randomized search took {(time() - start) / 60} minutes.")
    print('Best hyperparameters: ', rgrid.best_params_)
    return rgrid.best_params_

## Classifiers: Baseline performance 

### K-Nearest Neighbors (KNN) Classifier

In [ ]:
print('KNN Classifier: Baseline performance\n')
fit_eval_clf(KNeighborsClassifier(n_jobs=-1))

### Support Vector Classifier

In [ ]:
print('Support Vector: Baseline performance\n')
fit_eval_clf(SVC(random_state=321))

### Logistic Regression

In [ ]:
print('Logistic Regression: Baseline performance\n')
fit_eval_clf(LogisticRegression(random_state=321))

### Random Forest

In [ ]:
print('Random Forest: Baseline performance\n')
fit_eval_clf(RandomForestClassifier(n_jobs=-1,random_state=321))

## Classifiers: Tune hyperparameters

### Tune K-Nearest Neighbors (KNN) Classifier

In [ ]:
# Construct the set of hyperparameters to tune
params = {'n_neighbors':[5,6,7],
          'leaf_size':[1,2,3],
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree']}
best_knn = rand_search_cv(KNeighborsClassifier(), params)

In [ ]:
print('KNN Classifier: Performance after tuning hyperparameters\n')
fit_eval_clf(KNeighborsClassifier(**best_knn, n_jobs=-1))

### Tune Support Vector

In [ ]:
# Construct the set of hyperparameters to tune
params = {'C': [0.01,0.1,1,3], 
          'gamma': [0.01, 0.1, 1]}
best_sv = rand_search_cv(SVC(), params)

In [ ]:
print('Support Vector: Performance after tuning hyperparameters\n')
fit_eval_clf(SVC(**best_sv, random_state=321))

### Tune Logistic Regression

In [ ]:
# Construct the set of hyperparameters to tune
params = {'C': [0.01, 0.1, 1, 5, 10],
          'penalty':['l2']}
best_lr = rand_search_cv(LogisticRegression(random_state=321), params)

In [ ]:
print('Logistic Regression: Performance after tuning hyperparameters\n')
fit_eval_clf(LogisticRegression(**best_lr, random_state=321))

### Tune Random Forest

In [ ]:
# Construct the set of hyperparameters to tune
params = {'max_depth': [ 10, 15, 20, 30, 40],
          'min_samples_leaf': [1, 3, 4, 7],
          'min_samples_split': [2, 5, 7, 9],
          'n_estimators': [100, 200, 500, 700]}
best_rf = rand_search_cv(RandomForestClassifier(), params)

In [ ]:
print('Random Forest: Performance after tuning hyperparameters \n')
fit_eval_clf(RandomForestClassifier(**best_rf, n_jobs=-1, random_state=321))

In [ ]:
# Feature importances
model = RandomForestClassifier(**best_rf, n_jobs=-1, random_state=321)
model.fit(X_train_rus,y_train_rus)

feat_imp = pd.Series(model.feature_importances_, index=x_kbest.columns).sort_values(ascending=False)

# Set figure size
fig = plt.figure(figsize=(12,9))
ax = fig.gca()

feat_imp.plot(kind='bar', title='Tuned Random Forest: Feature Importances', ax=ax)
plt.ylabel('Feature Importance Score')
plt.tight_layout()
plt.show()

## Ensemble method

###  Voting Classifier

In [ ]:
start = time()

# Tuned Classifiers
knn = KNeighborsClassifier(**best_knn,n_jobs=-1)
svc = SVC(**best_sv, probability=True)
logit = LogisticRegression(**best_lr, random_state=321)
rfc = RandomForestClassifier(**best_rf, n_jobs=-1, random_state=321)

# Create the ensemble model
voting_clf = VotingClassifier(estimators=[('KNN', knn), ('Support Vector', svc),('Logistic Regression', logit),
                                          ('Random Forest', rfc)], voting='soft', weights=[1,2,2,2])

#Fit the model 
prediction=voting_clf.fit(X_train_rus, y_train_rus).predict(X_test)

# Accuracy score on test samples
print ("Accuracy score on test samples: ", round(accuracy_score (y_test, prediction), 4))
    
# Area under ROC on test samples
print ("Area under ROC curve on test samples: ", round(roc_auc_score (y_test, prediction), 4))
print()
print(f"Total time to run: {(time() - start) / 60:.2} minutes.")

### Boosting method 1: Gradient boosting

In [ ]:
print('Gradient boosting classifier\n')
fit_eval_clf(GradientBoostingClassifier(random_state=321))

### Boosting method 2: Adaptive boosting 

In [ ]:
print('Adaptive boosting classifier\n')
fit_eval_clf(AdaBoostClassifier(random_state=321))

### Stacking

In [ ]:
print('Stacking classifier\n')
start=time()

# Stacking Classifier
stacked_clf = StackingClassifier(classifiers=[knn, svc, rfc], meta_classifier=logit)

#Fit the model 
prediction=stacked_clf.fit(X_train_rus, y_train_rus).predict(X_test)

# Accuracy score on test samples
print ("Accuracy score on test samples: ", round(accuracy_score (y_test, prediction), 4))
    
# Area under ROC on test samples
print ("Area under ROC curve on test samples: ", round(roc_auc_score (y_test, prediction), 4))
print()
print(f"Total time to run: {(time() - start) / 60:.2} minutes.")


## Plot performances

In [ ]:
# Plot performances of baseline and tuned classifiers
clf_perform = pd.DataFrame({'Classifier': ['KNN', 'Support Vector', 'Logistic Regression', 'Random Forest'],
                            'Accuracy_Baseline': [0.5667, 0.6245, 0.6046, 0.5994],
                            'Accuracy_Tuned': [0.5743, 0.6238, 0.6051, 0.6389],
                            'AreaunderROC_Baseline': [0.5694, 0.6068, 0.5986, 0.5652], 
                            'AreaunderROC_Tuned': [0.5748, 0.6085, 0.5998, 0.6115]})
# Set figure size
fig = plt.figure(figsize = (10,8))
ax = fig.gca()

clf_perform.plot(kind='bar', x='Classifier', title='Classifier performance', ylim=[0.4,0.7], ax=ax)
plt.xticks(rotation=35)
plt.ylabel('Score')
plt.tight_layout()
plt.show()

In [ ]:
# Plot performances of ensemble classifiers
ensemble_perform = pd.DataFrame({'Ensemble': ['Bagging_Voting', 'Gradient boosting', 'Adaptive boosting', 
                                              'Stacking_Classifier'],
                                 'Accuracy': [0.6221, 0.6357, 0.6285, 0.5743],
                                 'Areaunder_ROC': [0.6105, 0.6144, 0.6107, 0.5748]})
# Set figure size
fig = plt.figure(figsize = (10,8))
ax = fig.gca()

ensemble_perform.plot(x='Ensemble',kind='bar',ylim=[0.4,0.7], title='Ensemble Classifiers Performance', ax=ax)
plt.ylabel('Score')
plt.xticks(rotation=23)
plt.tight_layout()
plt.show()

In [ ]:
print(clf_perform)
print(ensemble_perform)

## Results

* 最重要的 feature 是 Number of vehicles involved, location (latitude and longitude), speed limit (60mph), urban location and number of casualties